In [1]:
import tensorflow as tf
from tensorflow.keras import layers, models
import json
import os
import shutil
from tensorflow.keras.optimizers import Adam
import pandas as pd
import numpy as np
import random


In [4]:
with open('./kaggle.json') as f:
    kaggle_credentials = json.load(f)

    os.environ['KAGGLE_USERNAME'] = kaggle_credentials['username']
    os.environ['KAGGLE_KEY'] = kaggle_credentials['key']

    from kaggle.api.kaggle_api_extended import KaggleApi
    api = KaggleApi()
    api.authenticate()
    api.dataset_download_files("hitman1309/isic-2018-task-3", unzip=True)

Dataset URL: https://www.kaggle.com/datasets/hitman1309/isic-2018-task-3


In [3]:
def create_folder(name,csv_path, images_folder):
  os.mkdir(name)
  os.mkdir(f'{name}/MEL')
  os.mkdir(f'{name}/NOTMEL')

  df = pd.read_csv(csv_path)
  images = df['image'].to_list()
  MEL = df['MEL'].to_list()

  for i,m in zip(images,MEL):
    if m == 1:
      shutil.copy(f'{images_folder}/{i}.jpg',f'{name}/MEL')
    else:
      shutil.copy(f'{images_folder}/{i}.jpg',f'{name}/NOTMEL')

  notmel = os.listdir(f'{name}/NOTMEL')
  random.shuffle(notmel)
  while len(os.listdir(f'{name}/NOTMEL')) > len(os.listdir(f'{name}/MEL')):
    os.remove(f'{name}/NOTMEL/{notmel.pop()}')

In [4]:
create_folder('train','/content/ISIC2018_Task3_Training_GroundTruth/ISIC2018_Task3_Training_GroundTruth/ISIC2018_Task3_Training_GroundTruth.csv','/content/ISIC2018_Task3_Training_Input/ISIC2018_Task3_Training_Input')


In [7]:
create_folder('val','/content/ISIC2018_Task3_Validation_GroundTruth/ISIC2018_Task3_Validation_GroundTruth/ISIC2018_Task3_Validation_GroundTruth.csv','/content/ISIC2018_Task3_Validation_Input/ISIC2018_Task3_Validation_Input')


In [37]:
from tensorflow.keras.preprocessing import image_dataset_from_directory


train_dataset = image_dataset_from_directory(
    "train",
    image_size=(224, 224),
    batch_size=32
)

num_classes = len(train_dataset.class_names)

val_dataset = image_dataset_from_directory(
    "val",
    image_size=(224, 224),
    batch_size=32
)


Found 2226 files belonging to 2 classes.
Found 42 files belonging to 2 classes.


In [38]:
from tensorflow.keras.applications.resnet50 import preprocess_input

AUTOTUNE = tf.data.AUTOTUNE

def preprocess(ds):
    return ds.map(lambda x, y: (preprocess_input(x), y), num_parallel_calls=AUTOTUNE)

train_dataset = preprocess(train_dataset).prefetch(buffer_size=AUTOTUNE)
val_dataset = preprocess(val_dataset).prefetch(buffer_size=AUTOTUNE)


In [39]:
from tensorflow.keras.applications import ResNet50
from tensorflow.keras import layers, models

base_model = ResNet50(weights='imagenet', include_top=False, input_shape=(224, 224, 3))
base_model.trainable = False  # Para fine-tuning posterior

model = models.Sequential([
    base_model,
    layers.GlobalAveragePooling2D(),
    layers.Dense(128, activation='relu'),
    layers.Dropout(0.3),
    layers.Dense(num_classes, activation='softmax' if num_classes > 1 else 'sigmoid')
])


In [40]:
model.compile(
    optimizer='adam',
    loss='sparse_categorical_crossentropy' if num_classes > 1 else 'binary_crossentropy',
    metrics=['accuracy']
)


In [41]:
history = model.fit(
    train_dataset,
    validation_data=val_dataset,
    epochs=10,
    callbacks=[
        tf.keras.callbacks.ModelCheckpoint("best_model.keras", save_best_only=True),
        tf.keras.callbacks.EarlyStopping(patience=3, restore_best_weights=True)
    ]
)

Epoch 1/10
70/70 ━━━━━━━━━━━━━━━━━━━━ 36s 344ms/step - accuracy: 0.6840 - loss: 0.7436 - val_accuracy: 0.7857 - val_loss: 0.5018
Epoch 2/10
70/70 ━━━━━━━━━━━━━━━━━━━━ 27s 200ms/step - accuracy: 0.8016 - loss: 0.4246 - val_accuracy: 0.8095 - val_loss: 0.5024
Epoch 3/10
70/70 ━━━━━━━━━━━━━━━━━━━━ 26s 280ms/step - accuracy: 0.8146 - loss: 0.3869 - val_accuracy: 0.8095 - val_loss: 0.4333
Epoch 4/10
70/70 ━━━━━━━━━━━━━━━━━━━━ 15s 209ms/step - accuracy: 0.8275 - loss: 0.3674 - val_accuracy: 0.7143 - val_loss: 0.5157
Epoch 5/10
70/70 ━━━━━━━━━━━━━━━━━━━━ 20s 203ms/step - accuracy: 0.8393 - loss: 0.3530 - val_accuracy: 0.7857 - val_loss: 0.4335
Epoch 6/10
70/70 ━━━━━━━━━━━━━━━━━━━━ 21s 205ms/step - accuracy: 0.8425 - loss: 0.3437 - val_accuracy: 0.6667 - val_loss: 0.5666


In [56]:
base_model.trainable = True
model.compile(optimizer=tf.keras.optimizers.Adam(1e-5),
              loss='sparse_categorical_crossentropy',
              metrics=['accuracy'])

model.fit(train_dataset, validation_data=val_dataset, epochs=5)


Epoch 1/5
70/70 ━━━━━━━━━━━━━━━━━━━━ 36s 322ms/step - accuracy: 0.8302 - loss: 0.3622 - val_accuracy: 0.8095 - val_loss: 0.4355
Epoch 2/5
70/70 ━━━━━━━━━━━━━━━━━━━━ 14s 203ms/step - accuracy: 0.8294 - loss: 0.3472 - val_accuracy: 0.8095 - val_loss: 0.4439
Epoch 3/5
70/70 ━━━━━━━━━━━━━━━━━━━━ 14s 202ms/step - accuracy: 0.8486 - loss: 0.3289 - val_accuracy: 0.8095 - val_loss: 0.4542
Epoch 4/5
70/70 ━━━━━━━━━━━━━━━━━━━━ 15s 213ms/step - accuracy: 0.8419 - loss: 0.3439 - val_accuracy: 0.8095 - val_loss: 0.4628
Epoch 5/5
70/70 ━━━━━━━━━━━━━━━━━━━━ 20s 202ms/step - accuracy: 0.8507 - loss: 0.3311 - val_accuracy: 0.7857 - val_loss: 0.4700


In [17]:
create_folder('test','/content/ISIC2018_Task3_Test_GroundTruth/ISIC2018_Task3_Test_GroundTruth/ISIC2018_Task3_Test_GroundTruth.csv','/content/ISIC2018_Task3_Test_Input/ISIC2018_Task3_Test_Input')

In [57]:
test_dataset = image_dataset_from_directory(
    "test",
    image_size=(224, 224),
    batch_size=32,
    shuffle=False
)
test_dataset = preprocess(test_dataset).prefetch(buffer_size=AUTOTUNE)


Found 342 files belonging to 2 classes.


In [58]:
test_loss, test_accuracy = model.evaluate(test_dataset)

# Imprimir las métricas
print(f"Test Loss: {test_loss}")
print(f"Test Accuracy: {test_accuracy}")

11/11 ━━━━━━━━━━━━━━━━━━━━ 4s 340ms/step - accuracy: 0.8287 - loss: 0.4295
Test Loss: 0.4568873941898346
Test Accuracy: 0.780701756477356


In [51]:
# Cargar el mejor modelo guardado
best_model = tf.keras.models.load_model("best_model.keras")

# Evaluar el modelo en el conjunto de prueba
test_loss, test_accuracy = best_model.evaluate(test_dataset)

# Imprimir las métricas
print(f"Test Loss: {test_loss}")
print(f"Test Accuracy: {test_accuracy}")


11/11 ━━━━━━━━━━━━━━━━━━━━ 10s 316ms/step - accuracy: 0.8301 - loss: 0.4037
Test Loss: 0.45910483598709106
Test Accuracy: 0.7660818696022034


In [52]:
from sklearn.metrics import f1_score, confusion_matrix, classification_report


predictions = best_model.predict(test_dataset)
predictions = tf.argmax(predictions, axis=1)

true_labels = []
for image, label in test_dataset:
    true_labels.extend(label.numpy())

conf_matrix = confusion_matrix(true_labels, predictions)
print("Confusion Matrix:")
print(conf_matrix)

class_report = classification_report(true_labels, predictions)
print("Classification Report:")
print(class_report)


11/11 ━━━━━━━━━━━━━━━━━━━━ 11s 546ms/step
Confusion Matrix:
[[150  21]
 [ 59 112]]
Classification Report:
              precision    recall  f1-score   support

           0       0.72      0.88      0.79       171
           1       0.84      0.65      0.74       171

    accuracy                           0.77       342
   macro avg       0.78      0.77      0.76       342
weighted avg       0.78      0.77      0.76       342



In [54]:
from sklearn.metrics import f1_score, confusion_matrix, classification_report


predictions = model.predict(test_dataset)
predictions = tf.argmax(predictions, axis=1)

true_labels = []
for image, label in test_dataset:
    true_labels.extend(label.numpy())

conf_matrix = confusion_matrix(true_labels, predictions)
print("Confusion Matrix:")
print(conf_matrix)

class_report = classification_report(true_labels, predictions)
print("Classification Report:")
print(class_report)


11/11 ━━━━━━━━━━━━━━━━━━━━ 4s 315ms/step
Confusion Matrix:
[[143  28]
 [ 38 133]]
Classification Report:
              precision    recall  f1-score   support

           0       0.79      0.84      0.81       171
           1       0.83      0.78      0.80       171

    accuracy                           0.81       342
   macro avg       0.81      0.81      0.81       342
weighted avg       0.81      0.81      0.81       342



In [55]:
model.save("classifier.keras")